# Basic Kernel
https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/model_agnostic/Simple%20Kernel%20SHAP.html

In [74]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import itertools
import scipy.special
import time
import shap
import pandas as pd

# Step 1: Generate a simple regression dataset
X, y = make_regression(n_samples=1000, n_features=10, noise=0.5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Train a RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Step 4: Select the first test instance for explanation
test_instance = X_test[0]

# Define helper functions for Kernel SHAP
def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s) + 1))

def shapley_kernel(M, s):
    if s == 0 or s == M:
        return 10000  # Large constant for numerical stability
    return (M - 1) / (scipy.special.binom(M, s) * s * (M - s))

def kernel_shap(f, x, reference, M):
    X = np.zeros((2**M, M + 1))
    X[:, -1] = 1
    weights = np.zeros(2**M)
    V = np.zeros((2**M, M))
    for i in range(2**M):
        V[i, :] = reference

    for i, s in enumerate(powerset(range(M))):
        s = list(s)
        V[i, s] = x[s]
        X[i, s] = 1
        weights[i] = shapley_kernel(M, len(s))
    y = f(V)
    wsq = np.sqrt(weights)
    result = np.linalg.lstsq(wsq[:, None] * X, wsq * y, rcond=None)[0]
    return result

# Define the prediction function
def prediction_function(X):
    return regressor.predict(X)

# Define the reference input (mean of the training data)
reference = np.mean(X_train, axis=0)

# Number of features
M = X_train.shape[1]

# Step 6: Compute SHAP values and time the computation

# TreeExplainer SHAP
start_time = time.time()
tree_explainer = shap.TreeExplainer(regressor)
tree_shap_values = tree_explainer.shap_values(test_instance.reshape(1, -1))[0]
tree_baseline = float(tree_explainer.expected_value)  # Ensure scalar
tree_time = time.time() - start_time

# KernelExplainer SHAP
start_time = time.time()
kernel_explainer = shap.KernelExplainer(prediction_function, X_train)
kernel_shap_values = kernel_explainer.shap_values(test_instance.reshape(1, -1))[0]
kernel_baseline = float(kernel_explainer.expected_value)  # Ensure scalar
kernel_time = time.time() - start_time

# Custom Kernel SHAP
start_time = time.time()
custom_kernel_phi = kernel_shap(prediction_function, test_instance, reference, M)
custom_baseline = float(custom_kernel_phi[-1])  # Baseline (intercept), ensure scalar
custom_kernel_shap_values = custom_kernel_phi[:-1]  # SHAP values for features
custom_kernel_time = time.time() - start_time  # Time taken for Custom Kernel SHAP

# Step 7: Display Results
results = pd.DataFrame({
    "Feature": [f"Feature {i+1}" for i in range(X.shape[1])],
    "TreeExplainer SHAP": np.round(tree_shap_values, 4),  # TreeExplainer output
    "KernelExplainer SHAP": np.round(kernel_shap_values, 4),  # KernelExplainer output
    "CustomKernel SHAP": np.round(custom_kernel_shap_values, 4),  # Custom Kernel SHAP
})

# Add baselines to the results
results.loc["Baseline"] = {
    "Feature": "Baseline",
    "TreeExplainer SHAP": np.round(tree_baseline, 4),
    "KernelExplainer SHAP": np.round(kernel_baseline, 4),
    "CustomKernel SHAP": np.round(custom_baseline, 4),
}

# Calculate the sum of SHAP values + baseline for each method
tree_shap_sum = float(np.sum(tree_shap_values) + tree_baseline)  # Ensure scalar
kernel_shap_sum = float(np.sum(kernel_shap_values) + kernel_baseline)  # Ensure scalar
custom_shap_sum = float(np.sum(custom_kernel_shap_values) + custom_baseline)  # Ensure scalar

results.loc["Sum"] = {
    "Feature": "Sum",
    "TreeExplainer SHAP": np.round(tree_shap_sum, 4),
    "KernelExplainer SHAP": np.round(kernel_shap_sum, 4),
    "CustomKernel SHAP": np.round(custom_shap_sum, 4),
}

# Add computation times as the last row
results.loc["Computation Time (s)"] = {
    "Feature": "Computation Time (s)",
    "TreeExplainer SHAP": np.round(tree_time, 4),
    "KernelExplainer SHAP": np.round(kernel_time, 4),
    "CustomKernel SHAP": np.round(custom_kernel_time, 4),
}

results

C:\Users\mnoorche\AppData\Local\Temp\ipykernel_21016\4159486803.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tree_baseline = float(tree_explainer.expected_value)  # Ensure scalar
Using 800 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1 [00:00<?, ?it/s]

,Feature,TreeExplainer SHAP,KernelExplainer SHAP,CustomKernel SHAP
0,Feature 1,13.4197,13.4311,12.4019
1,Feature 2,1.7568,1.3560,3.4165
2,Feature 3,1.9600,0.9772,-0.5405
3,Feature 4,75.3594,74.8056,72.5709
4,Feature 5,1.0264,0.5648,-0.5133
5,Feature 6,-3.2434,-3.4140,-3.5330
6,Feature 7,-30.7180,-31.2188,-33.8595
7,Feature 8,1.8544,1.8552,1.8361
8,Feature 9,1.4149,0.8683,0.1188
9,Feature 10,-71.4951,-68.8819,-69.9190


# Updated Kernel

In [92]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import itertools
import scipy.special
import time
import shap
import pandas as pd

# Step 1: Generate a simple regression dataset
X, y = make_regression(n_samples=1000, n_features=10, noise=0.5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Train a RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Step 4: Select the first test instance for explanation
test_instance = X_test[0]

# Monte Carlo-based sampling for subsets
def sample_subsets(M, num_samples):
    """
    Generate random subsets of features using Monte Carlo sampling.
    This replaces the exhaustive powerset generation for efficiency.
    """
    np.random.seed(0)
    subsets = []
    for _ in range(num_samples):
        subset = np.random.choice([0, 1], size=M, p=[0.5, 0.5])
        subsets.append(np.where(subset == 1)[0])  # Indices of features in the subset
    return subsets

# Shapley kernel weight calculation
def shapley_kernel(M, s):
    """
    Compute the Shapley kernel weight for a given subset size `s`.
    """
    if s == 0 or s == M:
        return 10000  # Large constant for numerical stability
    return (M - 1) / (scipy.special.binom(M, s) * s * (M - s))

# Kernel SHAP implementation with optional Monte Carlo sampling
def kernel_shap(f, x, reference, M, num_samples=None):
    """
    Compute SHAP values using the Kernel SHAP method.
    - If `num_samples` is None, compute exact SHAP values (2^M subsets).
    - Otherwise, approximate SHAP values using Monte Carlo sampling.

    Args:
        f (callable): Prediction function.
        x (np.ndarray): Input instance to explain.
        reference (np.ndarray): Reference values for each feature.
        M (int): Number of features.
        num_samples (int): Number of Monte Carlo samples (optional).

    Returns:
        np.ndarray: SHAP values for the input instance.
    """
    if num_samples is None:
        # Use exact computation (exhaustive powerset)
        subsets = list(itertools.chain.from_iterable(itertools.combinations(range(M), r) for r in range(M + 1)))
    else:
        # Use Monte Carlo sampling
        subsets = sample_subsets(M, num_samples)

    # Prepare matrices for least-squares regression
    num_subsets = len(subsets)
    X = np.zeros((num_subsets, M + 1))
    X[:, -1] = 1  # Bias term
    V = np.tile(reference, (num_subsets, 1))
    weights = np.zeros(num_subsets)

    # Populate X, V, and weights based on subsets
    for i, s in enumerate(subsets):
        s = list(s)
        V[i, s] = x[s]
        X[i, s] = 1
        weights[i] = shapley_kernel(M, len(s))

    # Normalize weights for numerical stability
    weights /= np.sum(weights)

    # Model predictions for perturbed inputs
    y = f(V)

    # Weighted least-squares regression
    wsq = np.sqrt(weights)
    result = np.linalg.lstsq(wsq[:, None] * X, wsq * y, rcond=None)[0]

    return result


# Define the prediction function
def prediction_function(X):
    return regressor.predict(X)

# Define the reference input (mean of the training data)
reference = np.mean(X_train, axis=0)

# Number of features
M = X_train.shape[1]

# Step 6: Compute SHAP values and time the computation

# TreeExplainer SHAP
start_time = time.time()
tree_explainer = shap.TreeExplainer(regressor)
tree_shap_values = tree_explainer.shap_values(test_instance.reshape(1, -1))[0]
tree_baseline = float(tree_explainer.expected_value)  # Ensure scalar
tree_time = time.time() - start_time

# KernelExplainer SHAP
start_time = time.time()
kernel_explainer = shap.KernelExplainer(prediction_function, np.tile(reference, (500, 1)))
kernel_shap_values = kernel_explainer.shap_values(test_instance.reshape(1, -1))[0]
kernel_baseline = float(kernel_explainer.expected_value)  # Ensure scalar
kernel_time = time.time() - start_time

# Custom Kernel SHAP
start_time = time.time()
num_samples = 100  # Increase Monte Carlo samples for accuracy
custom_kernel_phi = kernel_shap(prediction_function, test_instance, reference, M, num_samples)
custom_baseline = float(custom_kernel_phi[-1])  # Baseline (intercept), ensure scalar
custom_kernel_shap_values = custom_kernel_phi[:-1]  # SHAP values for features
custom_kernel_time = time.time() - start_time  # Time taken for Custom Kernel SHAP

# Step 7: Display Results
results = pd.DataFrame({
    "Feature": [f"Feature {i+1}" for i in range(X.shape[1])],
    "TreeExplainer SHAP": np.round(tree_shap_values, 4),  # TreeExplainer output
    "KernelExplainer SHAP": np.round(kernel_shap_values, 4),  # KernelExplainer output
    "CustomKernel SHAP": np.round(custom_kernel_shap_values, 4),  # Custom Kernel SHAP
})

# Add baselines to the results
results = pd.concat([
    results,
    pd.DataFrame([
        {"Feature": "Baseline", "TreeExplainer SHAP": np.round(tree_baseline, 4),
         "KernelExplainer SHAP": np.round(kernel_baseline, 4),
         "CustomKernel SHAP": np.round(custom_baseline, 4)},
        {"Feature": "Sum", "TreeExplainer SHAP": np.round(np.sum(tree_shap_values) + tree_baseline, 4),
         "KernelExplainer SHAP": np.round(np.sum(kernel_shap_values) + kernel_baseline, 4),
         "CustomKernel SHAP": np.round(np.sum(custom_kernel_shap_values) + custom_baseline, 4)},
        {"Feature": "Computation Time (s)", "TreeExplainer SHAP": np.round(tree_time, 4),
         "KernelExplainer SHAP": np.round(kernel_time, 4),
         "CustomKernel SHAP": np.round(custom_kernel_time, 4)},
    ])
])

# Print the results
results

C:\Users\mnoorche\AppData\Local\Temp\ipykernel_21016\1762167479.py:117: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tree_baseline = float(tree_explainer.expected_value)  # Ensure scalar
Using 500 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1 [00:00<?, ?it/s]

,Feature,TreeExplainer SHAP,KernelExplainer SHAP,CustomKernel SHAP
0,Feature 1,-47.8522,-50.7102,-49.7264
1,Feature 2,9.0346,5.7863,9.8388
2,Feature 3,3.3038,-2.5937,0.9901
3,Feature 4,-2.1609,-4.1729,-17.3094
4,Feature 5,-0.7324,1.7503,-0.6865
5,Feature 6,-1.6299,-1.6725,4.1388
6,Feature 7,25.5404,14.7520,4.7303
7,Feature 8,35.0099,29.4719,30.4919
8,Feature 9,-74.1182,-83.5356,-88.8265
9,Feature 10,-115.6296,-82.9742,-79.2923


# mean with sampling

In [102]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import itertools
import scipy.special
import time
import shap
import pandas as pd


# Step 1: Generate a simple regression dataset
X, y = make_regression(n_samples=1000, n_features=5, noise=0.5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Step 2: Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Train a RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Step 4: Select the first test instance for explanation
test_instance = X_test[0]

# Shapley kernel weight calculation
def shapley_kernel(M, s):
    """
    Calculate the Shapley kernel weight for a subset of size s with M total features.

    Parameters:
    - M: Total number of features.
    - s: Size of the subset.

    Returns:
    - The kernel weight for the subset size.
    """
    if s == 0 or s == M:
        return 10000  # Large constant for numerical stability
    return (M - 1) / (scipy.special.binom(M, s) * s * (M - s))

# Mean Kernel SHAP function (Exact computation)
import numpy as np
import scipy.special

def mean_kernel_shap_with_constraint(f, x, reference, M, nsamples="auto"):
    r"""
    Kernel SHAP with additive efficiency constraint and kernel weights.
    
    Parameters:
    - f: The model function to explain.
    - x: Instance to explain (1D array of feature values).
    - reference: Reference value for each feature (1D array, usually the mean of the dataset).
    - M: Number of features.
    - nsamples: Number of samples for the sampling method. 
                If "auto", uses nsamples = min(2 * M + 2048, 2^M).
    
    Returns:
    - shap_values: Shapley values (1D array of size M).
    - baseline: The baseline value (\phi_0).
    """

    # Step 1: Calculate the number of samples
    if nsamples == "auto":
        nsamples = min(2 * M + 2048, 2 ** M)

    # Step 2: Sampling subsets with Shapley kernel weights
    np.random.seed(0)
    subsets = []  # List to store sampled subsets
    weights = []  # List to store corresponding weights

    for _ in range(nsamples):
        # Generate a random binary mask representing a subset
        subset = np.random.choice([0, 1], size=M, p=[0.5, 0.5])  # Random binary mask
        subset_indices = np.where(subset == 1)[0]
        subsets.append(subset_indices)
        
        # Compute Shapley kernel weight for the subset
        s_size = len(subset_indices)
        if s_size == 0 or s_size == M:
            weight = 10000  # Large constant for stability
        else:
            weight = (M - 1) / (scipy.special.binom(M, s_size) * s_size * (M - s_size))
        weights.append(weight)

    # Normalize weights
    weights = np.array(weights)
    weights /= np.sum(weights)

    # Number of subsets
    num_subsets = len(subsets)

    # Step 3: Initialize matrices for regression
    X = np.zeros((num_subsets, M))  # Design matrix
    V = np.zeros((num_subsets, M))  # Perturbed input data matrix

    # Prepare feature subsets
    for i, s in enumerate(subsets):
        V[i, :] = reference  # Start with reference values
        V[i, s] = x[s]       # Replace selected features with values from `x`
        X[i, s] = 1          # Set feature presence in the design matrix

    # Ensure `reference` and `x` are reshaped to (1, M) before passing to `f`
    reference = reference.reshape(1, -1)
    x = x.reshape(1, -1)

    # Step 4: Evaluate the model on the sampled feature subsets
    y = f(V) - f(reference)  # Centered outputs: v_x(s) - v_x(0)

    # Step 5: Add efficiency constraint row to X and y
    # Add a row to enforce the constraint: sum(ϕ_i) = v_x(1) - v_x(0)
    efficiency_row = np.ones((1, M))  # Row of ones
    X = np.vstack([X, efficiency_row])  # Append to the design matrix
    y = np.append(y, f(x) - f(reference))  # Append the efficiency constraint to outputs

    # Add corresponding weight for the efficiency constraint
    weights = np.append(weights, 1.0)  # Assign unit weight to the efficiency constraint

    # Step 6: Weighted least squares regression
    # Compute weighted least squares: Minimize Σ (w_i * (y_i - X_i^T * φ)^2)
    wsq = np.sqrt(weights)  # Square root of weights
    result = np.linalg.lstsq(wsq[:, None] * X, wsq * y, rcond=None)[0]  # Solve for SHAP values

    # Step 7: Return results
    return result, f(reference).flatten()


# Define the prediction function
def prediction_function(X):
    return regressor.predict(X)

# Number of features
M = X_train.shape[1]

# Step 6: Compute SHAP values and time the computation

# TreeExplainer SHAP
start_time = time.time()
tree_explainer = shap.TreeExplainer(regressor)
tree_shap_values = tree_explainer.shap_values(test_instance.reshape(1, -1))[0]
tree_baseline = float(tree_explainer.expected_value[0])  # Extract scalar from array
tree_time = time.time() - start_time

# KernelExplainer SHAP
start_time = time.time()
background = shap.sample(X_train, 100)  # Summarize the background to 100 samples
kernel_explainer = shap.KernelExplainer(prediction_function, background)
kernel_shap_values = kernel_explainer.shap_values(test_instance.reshape(1, -1))[0]
kernel_baseline = float(kernel_explainer.expected_value)  # Ensure scalar
kernel_time = time.time() - start_time

# Mean Kernel SHAP (Auto)
start_time = time.time()
reference = np.mean(X_train, axis=0)  # Mean feature values
mean_kernel_phi, mean_baseline = mean_kernel_shap_with_constraint(prediction_function, test_instance, reference, M)
mean_kernel_shap_values = mean_kernel_phi  # SHAP values for features
mean_kernel_time = time.time() - start_time

# Compute the prediction of the test_instance
test_instance_prediction = prediction_function(test_instance.reshape(1, -1))[0]

# Step 6: Display Results
results = pd.DataFrame({
    "Feature": [f"Feature {i+1}" for i in range(X.shape[1])],
    "TreeExplainer SHAP": np.round(tree_shap_values, 4),
    "KernelExplainer SHAP": np.round(kernel_shap_values, 4),
    "MeanKernel SHAP (auto)": np.round(mean_kernel_shap_values, 4),
})

# Append baseline, sum, computation time, and prediction
results = pd.concat([
    results,
    pd.DataFrame([
        {"Feature": "Baseline", "TreeExplainer SHAP": np.round(tree_baseline, 4),
         "KernelExplainer SHAP": np.round(kernel_baseline, 4),
         "MeanKernel SHAP (auto)": np.round(mean_baseline[0], 4)},
        {"Feature": "Sum", "TreeExplainer SHAP": np.round(np.sum(tree_shap_values) + tree_baseline, 4),
         "KernelExplainer SHAP": np.round(np.sum(kernel_shap_values) + kernel_baseline, 4),
         "MeanKernel SHAP (auto)": np.round(np.sum(mean_kernel_shap_values) + mean_baseline[0], 4)},
        {"Feature": "Computation Time (s)", "TreeExplainer SHAP": np.round(tree_time, 4),
         "KernelExplainer SHAP": np.round(kernel_time, 4),
         "MeanKernel SHAP (auto)": np.round(mean_kernel_time, 4)},
        {"Feature": "Prediction (Test Instance)", "TreeExplainer SHAP": np.round(test_instance_prediction, 4),
         "KernelExplainer SHAP": np.round(test_instance_prediction, 4), "MeanKernel SHAP (auto)": np.round(test_instance_prediction, 4)}
    ])
])


# Print the results
results

  0%|          | 0/1 [00:00<?, ?it/s]

,Feature,TreeExplainer SHAP,KernelExplainer SHAP,MeanKernel SHAP (auto)
0,Feature 1,21.0811,10.7461,24.1879
1,Feature 2,-7.5035,-8.2388,-15.5479
2,Feature 3,1.8309,2.0683,1.9465
3,Feature 4,-88.9366,-91.7532,-72.6232
4,Feature 5,-33.2918,-34.7344,-42.7720
0,Baseline,8.7755,23.8676,6.7641
1,Sum,-98.0444,-98.0444,-98.0446
2,Computation Time (s),0.0350,0.0450,0.0140
3,Prediction (Test Instance),-98.0444,-98.0444,-98.0444


# kernel based on game theory

In [104]:
import numpy as np
import math
import pandas as pd
from scipy.stats import mode

def shapley_kernel_weight(num_features, subset_size, normalize=True):
    """
    Calculate the Shapley kernel weight for a subset of a given size.

    Args:
        num_features (int): Total number of features.
        subset_size (int): Number of features in the subset.
        normalize (bool): Whether to normalize the weight to sum to 1.

    Returns:
        float: Weight for the subset size.
    """
    # Ensure subset size is valid
    if subset_size == 0 or subset_size == num_features:
        return 0

    # Original weight calculation
    weight = (num_features - 1) / (math.comb(num_features, subset_size) * subset_size * (num_features - subset_size))

    # Regularize weight to emphasize intermediate subset sizes
    intermediate_factor = 1 / (1 + abs(num_features / 2 - subset_size))
    weight *= intermediate_factor

    # Normalize weight if specified
    if normalize:
        total_combinations = 2 ** num_features - 2  # Exclude empty set and full set
        weight /= total_combinations

    return weight


def sample_and_weight(X, num_samples="auto", normalize=False, weighted_sampling=True):
    """
    Sample subsets and compute Shapley kernel weights.

    Args:
        X (np.ndarray): Input data, shape (n_samples, n_features).
        num_samples (int or str): Number of subsets to sample. Use "auto" to calculate based on the formula.
        normalize (bool): Whether to normalize the weights.
        weighted_sampling (bool): Whether to sample subsets based on weights.

    Returns:
        np.ndarray: Matrix of sampled subsets (binary masks).
        np.ndarray: Array of corresponding weights.
    """
    num_features = X.shape[1]

    # Set default number of samples
    if num_samples == "auto":
        num_samples = 2 * num_features + 2048

    subsets = []
    weights = []

    # Precompute weights for all subset sizes if using weighted sampling
    if weighted_sampling:
        subset_weights = [shapley_kernel_weight(num_features, size, normalize) for size in range(1, num_features)]
        cumulative_weights = np.cumsum(subset_weights)
        cumulative_weights /= cumulative_weights[-1]  # Normalize to sum to 1

    for _ in range(num_samples):
        if weighted_sampling:
            # Sample subset size based on weights
            rand = np.random.random()
            subset_size = np.searchsorted(cumulative_weights, rand) + 1
        else:
            # Uniformly sample subset size
            subset_size = np.random.randint(1, num_features)

        # Generate a binary mask for the subset
        subset = np.zeros(num_features, dtype=int)
        subset[:subset_size] = 1
        np.random.shuffle(subset)

        # Compute the Shapley kernel weight
        weight = shapley_kernel_weight(num_features, subset_size, normalize)

        subsets.append(subset)
        weights.append(weight)

    weights = np.array(weights)
    if normalize:
        weights /= np.sum(weights)  # Ensure weights sum to 1

    return np.array(subsets), weights



def prepare_data_subsets(sample_row, background_data, subsets, use_random=False):
    """
    Generate subsets with data substitution based on subsets and background data using matrix operations.
    A feature is treated as categorical if it has only 2 unique values (e.g., {0, 1}).

    Args:
        sample_row (pd.Series or np.ndarray): The single row of real data to explain.
        background_data (pd.DataFrame or np.ndarray): Background data used for substitution.
        subsets (np.ndarray): Binary subset matrix (1 = real data, 0 = background data).
        use_random (bool): 
            - If False, use mean (continuous) and mode (categorical) for substitution.
            - If True, randomly sample background data independently for each subset row.

    Returns:
        np.ndarray: Substituted data matrix where each row is a generated subset.
    """
    # Convert inputs to NumPy arrays if they are pandas DataFrames or Series
    if isinstance(sample_row, pd.Series):
        sample_row = sample_row.values
    if isinstance(background_data, pd.DataFrame):
        background_data = background_data.values

    num_features = background_data.shape[1]
    num_subsets = subsets.shape[0]

    # Step 1: Create a real value matrix by repeating the sample row
    real_value_matrix = np.tile(sample_row, (num_subsets, 1))

    # Step 2: Create background substitution matrix
    if use_random:
        # Random sampling: Generate a random sample for each row in subsets
        random_indices = np.random.randint(0, background_data.shape[0], size=num_subsets)
        background_substitution = background_data[random_indices, :]
    else:
        # Mean for continuous features and mode for categorical features
        background_substitution = []
        for i in range(num_features):
            unique_vals = np.unique(background_data[:, i])
            if len(unique_vals) == 2:  # If feature is categorical
                substitution_value = mode(background_data[:, i], keepdims=True).mode[0]
            else:  # Continuous
                substitution_value = np.mean(background_data[:, i])
            background_substitution.append(substitution_value)
        background_substitution = np.array(background_substitution)
        # Correctly repeat the substitution row for all subsets
        background_substitution = np.tile(background_substitution, (num_subsets, 1))

    # Step 3: Combine using matrix multiplication
    substituted_data = real_value_matrix * subsets + background_substitution * (1 - subsets)

    return substituted_data

# New Weighted Shapley Calculation Function
import numpy as np
import pandas as pd
from scipy.stats import mode

# Updated Weighted SHAP Function
def vectorized_weighted_shap_optimized(model, test_instance, background_data, num_samples="auto", use_random=True):
    """
    Optimized vectorized implementation of Shapley value calculation with adjustments for additivity.

    Args:
        model (callable): Prediction function of the trained model.
        test_instance (np.ndarray): Single test instance to explain.
        background_data (np.ndarray): Background data for substitution.
        num_samples (int or str): Number of samples. If "auto", use default 2 * M + 2048.
        use_random (bool): Whether to use random sampling for 0 values.

    Returns:
        np.ndarray: Adjusted Shapley values for each feature.
        float: Baseline prediction (average model prediction on background data).
    """
    num_features = test_instance.shape[0]

    # Step 1: Generate subsets and weights
    subsets, weights = sample_and_weight(background_data, num_samples=num_samples, weighted_sampling=True)

    # Ensure weights sum to 1
    weights = weights / np.sum(weights)
    assert np.isclose(weights.sum(), 1), "Weights are not properly normalized!"

    # Step 2: Create the substitution matrix
    test_instance_matrix = np.tile(test_instance, (subsets.shape[0], 1))

    # Generate background substitution values
    if use_random:
        random_indices = np.random.randint(0, background_data.shape[0], size=subsets.shape[0])
        background_samples = background_data[random_indices, :]
    else:
        background_mean_mode = np.mean(background_data, axis=0)
        background_samples = np.tile(background_mean_mode, (subsets.shape[0], 1))

    # Use subsets to replace 1s with test_instance and 0s with background_samples
    substituted_matrix = test_instance_matrix * subsets + background_samples * (1 - subsets)

    # Step 3: Compute model predictions in batches
    predictions = model(substituted_matrix)

    # Compute marginal contributions vectorized
    contributions = np.zeros((subsets.shape[0], num_features))
    for feature_idx in range(num_features):
        # Mask for subsets where the feature is excluded
        mask_without_feature = subsets[:, feature_idx] == 0

        # Create subset matrix where the feature is included
        subsets_with_feature = subsets[mask_without_feature].copy()
        subsets_with_feature[:, feature_idx] = 1

        # Generate the corresponding substitution matrix
        substituted_with_feature = test_instance_matrix[mask_without_feature] * subsets_with_feature + \
                                   background_samples[mask_without_feature] * (1 - subsets_with_feature)

        # Model predictions for v(S ∪ {X_i})
        predictions_with_feature = model(substituted_with_feature)

        # Marginal contributions: v(S ∪ {X_i}) - v(S)
        contributions[mask_without_feature, feature_idx] = predictions_with_feature - predictions[mask_without_feature]

    # Apply weights to contributions and compute weighted average
    shapley_values = np.average(contributions, axis=0, weights=weights)

    # Step 4: Compute baseline prediction
    baseline_prediction = model(background_data).mean()

    # Step 5: Enforce Additivity
    def enforce_additivity(shapley_values, baseline, prediction):
        """
        Adjust Shapley values to enforce additivity.

        Args:
            shapley_values (np.ndarray): Calculated Shapley values.
            baseline (float): Baseline prediction.
            prediction (float): Actual model prediction.

        Returns:
            np.ndarray: Adjusted Shapley values satisfying additivity.
        """
        total_contributions = np.sum(shapley_values)
        residual = prediction - (baseline + total_contributions)

        # Redistribute the residual proportionally across features
        shapley_values += residual * (shapley_values / total_contributions)

        return shapley_values

    # Get model prediction for the test instance
    prediction = model(test_instance.reshape(1, -1))[0]

    # Adjust Shapley values to satisfy additivity
    shapley_values = enforce_additivity(shapley_values, baseline_prediction, prediction)

    return shapley_values, baseline_prediction



# Add a new column to results
start_time = time.time()
weighted_shap_values, weighted_baseline = vectorized_weighted_shap_optimized(
    prediction_function, test_instance, X_train, num_samples="auto"
)
weighted_time = time.time() - start_time

# Add the new weighted SHAP method to the results DataFrame
results["Weighted SHAP"] = np.append(np.round(weighted_shap_values, 4), [
    np.round(weighted_baseline, 4),
    np.round(np.sum(weighted_shap_values) + weighted_baseline, 4),
    np.round(weighted_time, 4),
    np.round(test_instance_prediction, 4)
])

# Print the results DataFrame
results

,Feature,TreeExplainer SHAP,KernelExplainer SHAP,MeanKernel SHAP (auto),Weighted SHAP
0,Feature 1,21.0811,10.7461,24.1879,19.1302
1,Feature 2,-7.5035,-8.2388,-15.5479,-8.0885
2,Feature 3,1.8309,2.0683,1.9465,1.4556
3,Feature 4,-88.9366,-91.7532,-72.6232,-88.2724
4,Feature 5,-33.2918,-34.7344,-42.7720,-31.5424
0,Baseline,8.7755,23.8676,6.7641,9.2730
1,Sum,-98.0444,-98.0444,-98.0446,-98.0444
2,Computation Time (s),0.0350,0.0450,0.0140,0.0830
3,Prediction (Test Instance),-98.0444,-98.0444,-98.0444,-98.0444


# based on FastSHAP!

In [108]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time


@tf.keras.utils.register_keras_serializable()
def additivity_loss(y_true, y_pred):
    """
    Loss function with balanced regression loss and additivity constraint.

    Args:
        y_true: True SHAP values.
        y_pred: Predicted SHAP values.

    Returns:
        Regression loss with dynamically scaled additivity constraint.
    """
    shap_sum = tf.reduce_sum(y_pred, axis=1, keepdims=True)
    baseline = tf.reduce_mean(y_true, axis=1, keepdims=True)

    # Regression loss
    regression_loss = tf.reduce_mean(tf.square(y_true - y_pred))

    # Additivity constraint with reduced weight
    additivity_constraint = tf.reduce_mean(tf.square(shap_sum - baseline))
    return regression_loss + 0.1 * additivity_constraint  # Reduced weight


import numpy as np
from sklearn.cluster import KMeans

def select_representative_subset(X_train, n_clusters=10, subset_size=800, use_clustering=True):
    """
    Select a representative subset of the training data.
    
    If `use_clustering` is True, the subset is selected using clustering (KMeans). 
    Otherwise, random sampling without replacement is performed.

    Args:
        X_train (np.ndarray): Training data.
        n_clusters (int): Number of clusters for KMeans (used only if `use_clustering=True`).
        subset_size (int): Number of samples to select for the subset.
        use_clustering (bool): Whether to use clustering for subset selection.
    
    Returns:
        np.ndarray: Subset of training data.
    """
    if not use_clustering:
        # Random sampling without replacement
        subset_indices = np.random.choice(X_train.shape[0], size=subset_size, replace=False)
        return X_train[subset_indices]

    # Clustering-based subset selection
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(X_train)

    cluster_counts = np.bincount(cluster_labels)
    total_points = X_train.shape[0]

    samples_per_cluster = np.round((cluster_counts / total_points) * subset_size).astype(int)

    diff = subset_size - np.sum(samples_per_cluster)
    while diff != 0:
        for cluster_id in range(n_clusters):
            if diff == 0:
                break
            if diff > 0 and samples_per_cluster[cluster_id] < cluster_counts[cluster_id]:
                samples_per_cluster[cluster_id] += 1
                diff -= 1
            elif diff < 0 and samples_per_cluster[cluster_id] > 1:
                samples_per_cluster[cluster_id] -= 1
                diff += 1

    subset_indices = []
    for cluster_id in range(n_clusters):
        cluster_indices = np.where(cluster_labels == cluster_id)[0]
        if len(cluster_indices) <= samples_per_cluster[cluster_id]:
            subset_indices.extend(cluster_indices)
        else:
            subset_indices.extend(
                np.random.choice(cluster_indices, size=samples_per_cluster[cluster_id], replace=False)
            )

    subset_indices = np.array(subset_indices)
    return X_train[subset_indices]


def train_nn_on_weighted_shap(
    model, X_train, save_path="shap_nn_model.keras", n_clusters=10, subset_size=800, num_samples="auto"
):
    """
    Train a neural network using Weighted SHAP values from representative subsets.
    """
    X_subset = select_representative_subset(X_train, n_clusters, subset_size)

    shap_values_list = []
    baseline = np.mean(model(X_train))  # Average prediction for all background data

    for i in range(X_subset.shape[0]):
        shap_values, _ = vectorized_weighted_shap_optimized(
            model, X_subset[i], X_train, num_samples=num_samples, use_random=False
        )
        shap_values_list.append(shap_values)

    shap_values_array = np.array(shap_values_list)

    # Apply feature-wise scaling to SHAP values
    shap_scaler = StandardScaler()
    shap_values_normalized = shap_scaler.fit_transform(shap_values_array)

    # Create DataFrame for SHAP values and input features
    X_shap_df = pd.DataFrame(X_subset, columns=[f"Feature {i+1}" for i in range(X_subset.shape[1])])
    for i in range(shap_values_array.shape[1]):
        X_shap_df[f"SHAP Feature {i+1}"] = shap_values_array[:, i]

    # Train Neural Network
    num_features = X_train.shape[1]
    X_nn = X_shap_df[[f"Feature {i+1}" for i in range(num_features)]].values
    X_train_nn, X_val_nn, y_train_nn, y_val_nn = train_test_split(X_nn, shap_values_normalized, test_size=0.2, random_state=42)

    nn_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(num_features,)),
        tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(num_features, activation="linear")
    ])

    # Early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=additivity_loss, metrics=[tf.keras.metrics.MeanAbsoluteError()])
    nn_model.fit(X_train_nn, y_train_nn, validation_data=(X_val_nn, y_val_nn), epochs=300, batch_size=64, callbacks=[early_stopping], verbose=0)

    nn_model.save(save_path)

    return baseline, shap_scaler


def predict_nn_shap(model_path, test_instance, baseline, shap_scaler):
    """
    Predict SHAP values for a test instance using a saved neural network model.
    """
    nn_model = tf.keras.models.load_model(model_path)

    start_time = time.time()
    nn_shap_values_scaled = nn_model.predict(test_instance.reshape(1, -1)).flatten()
    nn_shap_values = shap_scaler.inverse_transform(nn_shap_values_scaled.reshape(1, -1)).flatten()
    prediction_time = time.time() - start_time

    # Adjust SHAP values to enforce additivity
    shap_sum = np.sum(nn_shap_values)
    adjustment = baseline - shap_sum
    nn_shap_values += adjustment / len(nn_shap_values)

    num_features = test_instance.shape[0]
    results = pd.DataFrame(
        {
            "Feature": [f"Feature {i+1}" for i in range(num_features)],
            "NN SHAP": nn_shap_values,
        }
    )
    results.loc[len(results.index)] = {"Feature": "Baseline", "NN SHAP": baseline}
    results.loc[len(results.index)] = {"Feature": "Prediction Time (s)", "NN SHAP": prediction_time}

    return results


# Example Workflow
baseline, shap_scaler = train_nn_on_weighted_shap(
    model=prediction_function,
    X_train=X_train,
    save_path="shap_nn_model.keras",
    n_clusters=1,
    subset_size=1000,
    num_samples="auto"
)

nn_results = predict_nn_shap(
    model_path="shap_nn_model.keras",
    test_instance=test_instance,
    baseline=baseline,
    shap_scaler=shap_scaler
)

# Update Results DataFrame
nn_shap_values = nn_results.loc[nn_results["Feature"].str.contains("Feature"), "NN SHAP"].values
nn_baseline = nn_results.loc[nn_results["Feature"] == "Baseline", "NN SHAP"].values[0]
results["NN SHAP"] = np.append(np.round(nn_shap_values, 4), [
    np.round(nn_baseline, 4),
    np.round(np.sum(nn_shap_values) + nn_baseline, 4),
    np.round(nn_results.loc[nn_results["Feature"] == "Prediction Time (s)", "NN SHAP"].values[0], 4),
    np.round(test_instance_prediction, 4)
])

results

C:\Users\mnoorche\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


,Feature,TreeExplainer SHAP,KernelExplainer SHAP,MeanKernel SHAP (auto),Weighted SHAP,NN SHAP
0,Feature 1,21.0811,10.7461,24.1879,19.1302,41.3835
1,Feature 2,-7.5035,-8.2388,-15.5479,-8.0885,12.5187
2,Feature 3,1.8309,2.0683,1.9465,1.4556,18.7043
3,Feature 4,-88.9366,-91.7532,-72.6232,-88.2724,-27.4381
4,Feature 5,-33.2918,-34.7344,-42.7720,-31.5424,-35.8953
0,Baseline,8.7755,23.8676,6.7641,9.2730,9.2730
1,Sum,-98.0444,-98.0444,-98.0446,-98.0444,18.5460
2,Computation Time (s),0.0350,0.0450,0.0140,0.0830,0.0430
3,Prediction (Test Instance),-98.0444,-98.0444,-98.0444,-98.0444,-98.0444
